This notebook contains additional pre-processing steps to prepare a single data file for further processing.

In [1]:
import os
import re
from difflib import SequenceMatcher

import pandas as pd
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
import nltk
import spacy

Specify path to input directory. Set home = 0 for WFH, 1 otherwise.

In [2]:
home = 0
if home:
    infiledir = r"C:\Possible_online_studies\NLP_expressive_writing\analysis\Processed_2"
    rand_dir = r"C:\Possible_online_studies\NLP_expressive_writing\analysis"
else:
    infiledir = r"P:\EW_analysis\analysis\Processed_2"
    rand_dir = r"P:\EW_analysis\analysis"

The input directory contains files for each of the baseline and follow-up assessments (4 each). The reason for this is that different sets of images were used for the MST and the order was counter-balanced between participants.
We want to put everything into a single dataframe with an extra column providing information about the order.

In [3]:
def find_number(text): # this is just a function that allows us to extract numbers from strings. OP in str format.
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)

Apply the above function  to the specified columns of the 'randomisation' file. This file was used when assigning participants to the different groups and holds all the information about which baseline/follow-up assessments they were assigned to.

In [4]:
randomisation_df = pd.read_csv(os.path.join(rand_dir, 'randomisation.csv'))
randomisation_df.loc[:,['Participant number', 'Baseline type','Follow-up 1','Follow-up 2','Follow-up 3']] = randomisation_df.loc[:,['Participant number', 'Baseline type','Follow-up 1','Follow-up 2','Follow-up 3']].applymap(find_number).astype('float')
randomisation_df.rename(columns = {'Participant number': 'id'},inplace = True) # rename Participant number column and set to index
randomisation_df.set_index('id',inplace = True)

There were four different sequences participants could have been assigned to:
1 2 3 4;
2 1 4 3;
3 4 1 2;
4 3 2 1;
When assembling the dataframe and adding the order column, we need to take this into account when labelling the time points.

In [6]:
seq_1_ind = randomisation_df[(randomisation_df['Baseline type']==1) & (randomisation_df['Follow-up 1'] == 2)].index
seq_2_ind = randomisation_df[(randomisation_df['Baseline type']==2) & (randomisation_df['Follow-up 1'] == 1)].index
seq_3_ind = randomisation_df[(randomisation_df['Baseline type']==3) & (randomisation_df['Follow-up 1'] == 4)].index
seq_4_ind = randomisation_df[(randomisation_df['Baseline type']==4) & (randomisation_df['Follow-up 1'] == 3)].index

First of all we deal with the baseline files, because these are easy - we know that whatever the assessment number, the order number should always be '1'.

In [7]:
base_files = [f for f in os.listdir(infiledir) if 'base' in f] # use only those files with 'base' in the filename. Depends on filename so may need changing.
base_1 = pd.read_csv(os.path.join(infiledir, base_files[0]),index_col = 'id') # specify that we want to use id as the index to avoid having a generic index inserted
base_2 = pd.read_csv(os.path.join(infiledir, base_files[1]),index_col = 'id')
base_3 = pd.read_csv(os.path.join(infiledir, base_files[2]),index_col = 'id')
base_4 = pd.read_csv(os.path.join(infiledir, base_files[3]), index_col='id')

all_base_df = pd.concat([base_1,base_2,base_3,base_4],axis = 0) # now concatenate the above.
all_base_df['order'] = 1 # baseline will always be time point 1
all_base_df.reset_index(inplace = True)

Now for the follow-ups. The task type column is based on the MST task type, which caused nan values for participants with missing MST records. To avoid problems with these missing values, we replace them with the appropriate task number (note that the follow-up number indicates which set of MST images participants saw rather than the order of follow-up assessments).

In [8]:
fu_files = [f for f in os.listdir(infiledir) if 'follow' in f]
fu_1 = pd.read_csv(os.path.join(infiledir, fu_files[0]),index_col = 'id')
fu_1['task_type'].replace(np.nan,1,inplace = True)
fu_2 = pd.read_csv(os.path.join(infiledir, fu_files[1]),index_col = 'id')
fu_2['task_type'].replace(np.nan,2,inplace = True)
fu_3 = pd.read_csv(os.path.join(infiledir, fu_files[2]),index_col = 'id')
fu_3['task_type'].replace(np.nan,3,inplace = True)
fu_4 = pd.read_csv(os.path.join(infiledir, fu_files[3]), index_col='id')
fu_4['task_type'].replace(np.nan,4,inplace = True)

In [9]:
all_fu_df = pd.concat([fu_1,fu_2,fu_3,fu_4],axis = 0)
all_fu_df['order'] = np.nan # set this to nan so we can replace as appropriate

In [10]:
all_fu_df.loc[(all_fu_df.index.isin(seq_1_ind)) & (all_fu_df.task_type == 2),'order'] = 2
all_fu_df.loc[(all_fu_df.index.isin(seq_2_ind)) & (all_fu_df.task_type == 1),'order'] = 2
all_fu_df.loc[(all_fu_df.index.isin(seq_3_ind)) & (all_fu_df.task_type == 4),'order'] = 2
all_fu_df.loc[(all_fu_df.index.isin(seq_4_ind)) & (all_fu_df.task_type == 3),'order'] = 2

all_fu_df.loc[(all_fu_df.index.isin(seq_1_ind)) & (all_fu_df.task_type == 3),'order'] = 3
all_fu_df.loc[(all_fu_df.index.isin(seq_2_ind)) & (all_fu_df.task_type == 4),'order'] = 3
all_fu_df.loc[(all_fu_df.index.isin(seq_3_ind)) & (all_fu_df.task_type == 1),'order'] = 3
all_fu_df.loc[(all_fu_df.index.isin(seq_4_ind)) & (all_fu_df.task_type == 2),'order'] = 3

all_fu_df.loc[(all_fu_df.index.isin(seq_1_ind)) & (all_fu_df.task_type == 4),'order'] = 4
all_fu_df.loc[(all_fu_df.index.isin(seq_2_ind)) & (all_fu_df.task_type == 3),'order'] = 4
all_fu_df.loc[(all_fu_df.index.isin(seq_3_ind)) & (all_fu_df.task_type == 2),'order'] = 4
all_fu_df.loc[(all_fu_df.index.isin(seq_4_ind)) & (all_fu_df.task_type == 1),'order'] = 4


Participant 19 had a messed up order due to being sent the wrong mst task for assessment 2. We'll change the sequence here. We could just swap the number since we know where the problem is. However, since we don't want to rely on knowledge about the file (which someone else may not have, not having worked with it), we just check where the randomisation spreadsheet and the actual assessments don't match up. First, we find the number in the actual task files that doesn't match the randomisation. Then we check which of the randomisation numbers is not in the follow-ups. We then swap the former for the latter.

In [37]:
list_rand = list(randomisation_df.loc[19,['Follow-up 1','Follow-up 2','Follow-up 3']].values)
list_act = list(all_fu_df.loc[19,'task_type'].values)

swap_num = [f for f in list_act if f not in list_rand]
cor_num = [f for f in list_rand if f not in list_act]
all_fu_df.loc[19,'task_type'] = all_fu_df.loc[19,'task_type'].replace(swap_num, cor_num) # replace assessment number 1 with a 2

In [41]:
all_fu_df.loc[(all_fu_df.index==19) & (all_fu_df.task_type == 2),'order'] = 2 # now replace with the correct order number.
all_fu_df.loc[(all_fu_df.index==19) & (all_fu_df.task_type == 4),'order'] = 3
all_fu_df.loc[(all_fu_df.index==19) & (all_fu_df.task_type == 3),'order'] = 4

We'll have a look at any remaining NaN records. The '0' record is either left over from the piloting stage (since I use 0 as the participant ID for this), or someone entered their participant ID wrong. In any case, since no one was assigned Participant ID 0, we'll drop this record from the analysis. 73 seems to be missing the qualtrics record for the first follow-up, but has a complete record under order 1. We'll use the complete record as the one we'll process in the analysis below.

In [42]:
all_fu_df.drop(labels = 0, axis = 0, inplace = True) # drop the 0 record

In [43]:
# first we'll reset the index (because this makes it easier to drop the record we don't want and the duplicate indices for subject ids will also hinder us below)
all_fu_df.reset_index(inplace = True)
# now we drop the record we don't want
all_fu_df.drop(labels = all_fu_df.loc[(all_fu_df['id']==73) & (all_fu_df['task_type'] == 2)].index, axis = 0,inplace = True)


That's it! Now we can merge the dataframes into one big frame:

In [44]:
all_dat_df = all_base_df.append(all_fu_df,ignore_index = True)

Before looking at the data, we want to remove those subjects that have been excluded due to not completing enough of the study. 'Enough' here means at least the baseline, all three days of writing and one follow-up. We have a column in the randomisation data frame indicating 'excluded' status. We'll get the ids of affected subjects from this and then drop them from the main data frame.

In [45]:
Excluded_subs = randomisation_df.loc[:,'Excluded'].dropna().index
all_dat_df.drop(labels = all_dat_df[all_dat_df.id.isin(Excluded_subs)].index, axis = 0,inplace = True)

Let's check the proportion of missing values in the ADNM_burden_total column (our outcome variable).

In [50]:
print("The proportion of missing data is", all_dat_df.ADNM_burden_total.isna().sum()/all_dat_df.shape[0])

The proportion of missing data is 0.0


Let's check the number of 0s.

In [51]:
print("The proportion of 0s is", (all_dat_df.ADNM_burden_total==0).sum()/all_dat_df.shape[0])

The proportion of 0s is 0.03411513859275053


Zeros are meaningful so cannot be removed (they indicate that a participant no longer experienced symptoms of adjustment disorder at a given follow-up point).

In [128]:
#Participant #15 filled in the ADNM-20 quesetionnaire for task type 2 separately. We will have to score the survey and add the results here.
P15_ADNM = pd.read_csv(os.path.join(rand_dir, 'P15_task2_ADNM_20.csv'), skiprows=[0,2])
col_keys = pd.read_csv(os.path.join(rand_dir, 'survey_cols.csv'))

In [129]:
ADNM_cols = P15_ADNM.iloc[:,19:].columns
ADNM_items =  ADNM_cols[~ADNM_cols.str.contains('How long')&~ADNM_cols.str.contains('Below is a list of')&~ADNM_cols.str.contains('The events you have just')]

In [130]:
ADNM_preoccupation = ['I have to think about the stressful situation repeatedly.',
'I have to think about the stressful situation a lot and this is a great burden to me.',
"I constantly get memories of the stressful situation and can't do anything to stop them.",
'My thoughts often revolve around anything related to the stressful situation.']

ADNM_failure = ['Since the stressful situation, I find it difficult to concentrate on certain things.',
'Since the stressful situation, I do not like going to work or carrying out the necessary tasks in everyday life.',
'Since the situation, I can no longer sleep properly.',
'All in all, the situation causes serious impairment in my social or occupational life, my leisure time and other important areas of functioning.']

ADNM_depression = ['Since the stressful situation, I feel low and sad.',
'I rarely do those activities which I used to enjoy anymore.',
'I have been feeling dispirited since the stressful situation and have little hope for the future.']

ADNM_anxiety = ['If I think about the stressful situation, I find myself in a real state of anxiety.',
'Since the stressful situation, I am scared of doing certain things or of getting into certain situations.']

ADNM_impulse = ['I am nervous and restless since the stressful situation.',
'Since the stressful situation, I lose my temper much quicker than I used to, even over small things.',
'I have noticed that I am becoming more irritable due to the stressful situation.']

ADNM_avoidance = ['I try to avoid talking about the stressful situation whenever possible.',
'I avoid certain things that might remind me of the stressful situation.',
'I try to dismiss the stressful situation from my memory.',
'I try to suppress my feelings because they are a burden to me.']

ADNM_total = ADNM_preoccupation + ADNM_failure + ADNM_depression + ADNM_anxiety + ADNM_impulse +ADNM_avoidance

In [141]:
P15_ADNM.loc[:,ADNM_cols] = P15_ADNM.loc[:,ADNM_cols].replace({'Never':1, 'Rarely':2,'Sometimes':3,'Often':4,'<1 month':1,'1-3 months':2, '3-6 months':3, 'more than 6 months':4})
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_time_total']] = P15_ADNM.filter(like = 'How long have you had this reaction for?', axis = 1).sum(axis = 1).values
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_preoccupation']] = P15_ADNM.filter(items = ADNM_preoccupation,axis = 1).sum(axis = 1).values
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_failure']] = P15_ADNM.filter(items = ADNM_failure,axis = 1).sum(axis = 1).values
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_anxiety']] = P15_ADNM.filter(items = ADNM_anxiety,axis = 1).sum(axis = 1).values
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_depression']] = P15_ADNM.filter(items = ADNM_depression,axis = 1).sum(axis = 1).values
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_impulse']] = P15_ADNM.filter(items = ADNM_impulse,axis = 1).sum(axis = 1).values
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_avoidance']] = P15_ADNM.filter(items = ADNM_avoidance,axis = 1).sum(axis = 1).values
all_dat_df.loc[(all_dat_df.id == 15) & (all_dat_df.task_type == 2),['ADNM_burden_total']] = P15_ADNM.filter(items = ADNM_total,axis = 1).sum(axis = 1).values

Now let's check the ids with 0 scores one more time. We'll look at the task type variable too, so we can check what happened in the files.

In [142]:
all_dat_df.loc[all_dat_df.ADNM_burden_total == 0,['id','task_type']]

,id,task_type
177,85.0,1.0
226,11.0,2.0
294,98.0,2.0
298,302.0,2.0
299,134.0,2.0
328,46.0,3.0
335,1.0,3.0
338,10.0,3.0
381,104.0,3.0
396,1.0,4.0


72, task type 2 - didn't check 'other' in events questionnaire and wasn't shown rating scales, resulting in 0 score.
113 - didn't check events but supplied dates - wasn't shown rating scales
85,11,98,302,134,46,1,10,104,1,73, 90 - did not note experiencing any distressing events, so 0 score due to not being shown the rating scales.
23 - only supplied dates, wasn't shown rating scales


So now we know where the 0s occcur, and why. We can replace them with nans where appropriate so they don't affect the analysis in an undesirable way.

In [143]:
all_dat_df.loc[(all_dat_df.id.isin([72,113]))&(all_dat_df.task_type == 2),'ADNM_burden_total'] = np.nan
all_dat_df.loc[(all_dat_df.id==23)&(all_dat_df.task_type == 3),'ADNM_burden_total'] = np.nan

In [144]:
print("The proportion of missing data is", all_dat_df.ADNM_burden_total.isna().sum()/all_dat_df.shape[0]) # just check that the above worked as intended.
print("The proportion of 0s is", (all_dat_df.ADNM_burden_total == 0).sum()/all_dat_df.shape[0]) # just check that the above worked as intended.

The proportion of missing data is 0.006396588486140725
The proportion of 0s is 0.0255863539445629


In [145]:
all_dat_df.loc[all_dat_df.ADNM_burden_total.isna(),'id']

277     72.0
284    113.0
312     23.0
Name: id, dtype: float64

We now need to assess whether the events participants identified as the 'most stressful' in the ADNM 20 questionnaire are the same or similar at each time point. We are going to use two different ways to assess this, based on (i) the difflib sequence matcher and (ii) spacy. The latter assesses semantic similarity whereas the first will only give us the ratio of correspondence between two sequences.

In [ ]:
# rename column holding most stressful events.
all_dat_df.rename(columns = {all_dat_df.filter(like = 'The events',axis = 1).columns[0]: 'most_stressful_event'},inplace = True)

Approach based on sequence matcher. We preprocess the text in each cell by tokenizing, removing non-nouns/-verbs, making everything lower case and stemming. We also remove stop words and any words that are three or  fewer characters long. This is just to reduce the 'noise' in the sequences we compare. Note that you end up with a list of words for each participant, which is I used apply with list commprehension and a lambda. Don't judge please.

In [150]:
all_dat_df['most_stressful_event_nlp'] = all_dat_df.most_stressful_event
#tokenization using nltk (only apply to non-nan values in column); use regexptokenizer to remove punctuation marks also
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp']=all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'].apply(lambda x: tokenizer.tokenize(x))
# get only verbs and nouns.
all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp']=all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'].apply(lambda y:[f[0] for f in nltk.pos_tag(y) if any(x in f[1] for x in ['NN','VB'])])
# lower case everything
all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp']=all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'].apply(lambda x: [i.lower() for i in x])
# stemming 
ps = nltk.stem.PorterStemmer()
all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp']=all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'].apply(lambda x: [ps.stem(i) for i in x])
# remove stop words
stop_words = set(nltk.corpus.stopwords.words('english'))
all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'] = all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'].apply(lambda x: [i for i in x if not i in stop_words])
# remove all words <3 characters
all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'] = all_dat_df.loc[~all_dat_df.most_stressful_event.isna(),'most_stressful_event_nlp'].apply(lambda x: [i for i in x if len(i)>3])

In [151]:
# gives you the ratio of similar content between two sequences

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [152]:
all_dat_df['most_stressful_similarity'] = np.nan # create column to hold ratios for similarity between stressful events
for count,pnum in enumerate(all_dat_df.id):
    pnum_df = all_dat_df.loc[all_dat_df.id==pnum,:] # get entries for a single participant
    orig = all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==1),'most_stressful_event_nlp'].iloc[0] # OG stressful event (a.k.a. baseline most stressful event)
    for i,num in enumerate(all_dat_df.loc[(all_dat_df.id==pnum)&(all_dat_df.order>1),'order']):
        if all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==num),'most_stressful_event_nlp'].isna().iloc[0]:
            continue # skip if nan
        else: # else score similarity to baseline
            all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==num),'most_stressful_similarity'] = similar(orig, all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==num),'most_stressful_event_nlp'].iloc[0])

In [153]:
all_dat_df.loc[(all_dat_df['order']==1) & (all_dat_df.most_stressful_similarity.isna()),'most_stressful_similarity']=1 # set similarity of baseline to 1. This is just to avoid issues with nans or 0s later.

In [155]:
nlp = spacy.load('en_core_web_lg')
all_dat_df['most_stressful_similarity2'] = np.nan

for count,pnum in enumerate(all_dat_df.id):
    pnum_df = all_dat_df.loc[all_dat_df.id==pnum,:]
    orig = nlp(all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==1),'most_stressful_event'].iloc[0]) # OG stressful event (a.k.a. baseline most stressful event)
    for i,num in enumerate(all_dat_df.loc[(all_dat_df.id==pnum)&(all_dat_df.order>1),'order']):
      if all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==num),'most_stressful_event'].isna().iloc[0]:
        continue
      else:
        all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==num),'most_stressful_similarity2'] = orig.similarity(nlp(all_dat_df.loc[(all_dat_df['id']==pnum)&(all_dat_df['order']==num),'most_stressful_event'].iloc[0]))



In [156]:
# set baseline to 1 for second measure also
all_dat_df.loc[(all_dat_df['order']==1) & (all_dat_df.most_stressful_similarity2.isna()),'most_stressful_similarity2']=1

We will now add the group participants were assigned to as a column in the dataframe.

In [157]:
randomisation_df.filter(like = 'Group',axis = 1)
all_dat_df['Group'] = np.nan
EW_ids = randomisation_df[randomisation_df.Group == 'EW'].index
EWRE_ids = randomisation_df[randomisation_df.Group == 'EW+RE'].index
CTR_ids = randomisation_df[randomisation_df.Group == 'CT'].index
all_dat_df.loc[all_dat_df.id.isin(EW_ids),'Group'] = 'EW'
all_dat_df.loc[all_dat_df.id.isin(EWRE_ids),'Group'] = 'EWRE'
all_dat_df.loc[all_dat_df.id.isin(CTR_ids),'Group'] = 'CTR'

In [158]:
save = 1
if save:
    all_dat_df.to_csv(os.path.join(infiledir,'all_dat_df.csv'),index = False)